In [24]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
import numpy as np
from tensorflow.python.platform import gfile

In [3]:
model = keras.models.load_model('acc_81_aug_30')

In [6]:
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
BasicBlockCount (InputLayer)    [(None, 1)]          0                                            
__________________________________________________________________________________________________
BasicBlockWithSingleSuccessor ( [(None, 1)]          0                                            
__________________________________________________________________________________________________
BasicBlockWithTwoSuccessors (In [(None, 1)]          0                                            
__________________________________________________________________________________________________
BasicBlockWithMoreThanTwoSucces [(None, 1)]          0                                            
____________________________________________________________________________________________

In [21]:
import tensorflow as tf
import sys
from tensorflow.python.platform import gfile
from tensorflow.core.protobuf import saved_model_pb2
from tensorflow.python.util import compat

filename = 'acc_81_aug_30/saved_model.pb'
with gfile.FastGFile(filename, 'rb') as f:
    data = compat.as_bytes(f.read())
    sm = saved_model_pb2.SavedModel()
    sm.ParseFromString(data)
    g_in = tf.import_graph_def(sm.meta_graphs[0].graph_def)

In [10]:
# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(x=tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

In [ ]:

# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()

# inspect the layers operations inside your frozen graph definition and see the name of its input and output tensors
layers = [op.name for op in frozen_func.graph.get_operations()]
print("-" * 50)
print("Frozen model layers: ")
for layer in layers:
    print(layer)

print("-" * 50)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)

# Save frozen graph from frozen ConcreteFunction to hard drive
# serialize the frozen graph and its text representation to disk.

#Optional
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir="./frozen_models",
                  name="acc_81_aug_30_txt.pbtxt",
                as_text=True)

model.summary()

In [31]:
model = tf.keras.models.load_model('acc_81_aug_30')

In [32]:
type(model)

keras.engine.functional.Functional

In [34]:
tfmodel = tf.io.read_file('acc_81_aug_30/saved_model.pb')

In [35]:
type(tfmodel)

tensorflow.python.framework.ops.Tensor

In [36]:
tf.io.write_graph(
    tfmodel, 'logdir', 'tfmodel.pbtxt', as_text=True
)

AttributeError: 'Tensor' object has no attribute 'DESCRIPTOR'

In [18]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.
 
    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph

In [19]:
from keras import backend as K

# Create, compile and train model...

frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

TypeError: Keras symbolic inputs/outputs do not implement `op`. You may be trying to pass Keras symbolic inputs/outputs to a TF API that does not register dispatching, preventing Keras from automatically converting the API call to a lambda layer in the Functional Model.

In [37]:
from tensorflow.python.platform import gfile

filename = 'acc_81_aug_30/saved_model.pb'

with gfile.FastGFile(filename,'rb') as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())
    tf.import_graph_def(graph_def, name='')
    in_dir = os.path.dirname(filename)
    out_filename = os.path.splitext(os.path.basename(filename))[0] + ".pbtxt"
    tf.train.write_graph(graph_def, in_dir, out_filename, as_text=True)

DecodeError: Error parsing message

In [38]:
graphdef = tf.graph_util.import_graph_def('acc_81_aug_30/saved_model.pb')

TypeError: graph_def must be a GraphDef proto.

In [41]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
import numpy as np
#path of the directory where you want to save your model
frozen_out_path = 'test'
# name of the .pb file
frozen_graph_filename = "frozen_graph"
model = tf.keras.models.load_model('acc_81_aug_30')


In [43]:
# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: model(x))

In [46]:
model.inputs[0].shape

TensorShape([Dimension(None), Dimension(1)])

In [47]:
full_model = full_model.get_concrete_function(tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

ValueError: in user code:

    /tmp/ipykernel_5514/3905952325.py:2 None  *
        lambda x: model(x)
    /mnt/disks/data/tarindu/anaconda3/envs/llvm/lib/python3.9/site-packages/keras/engine/base_layer_v1.py:739 __call__  **
        input_spec.assert_input_compatibility(self.input_spec, inputs,
    /mnt/disks/data/tarindu/anaconda3/envs/llvm/lib/python3.9/site-packages/keras/engine/input_spec.py:199 assert_input_compatibility
        raise ValueError('Layer ' + layer_name + ' expects ' +

    ValueError: Layer model_6 expects 27 input(s), but it received 1 input tensors. Inputs received: [<tf.Tensor 'x:0' shape=(?, 1) dtype=float32>]


In [ ]:
# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()
layers = [op.name for op in frozen_func.graph.get_operations()]
print("-" * 60)
print("Frozen model layers: ")
for layer in layers:
    print(layer)
print("-" * 60)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)
# Save frozen graph to disk
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir=frozen_out_path,
                  name=f"{frozen_graph_filename}.pb",
                  as_text=False)
# Save its text representation
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir=frozen_out_path,
                  name=f"{frozen_graph_filename}.pbtxt",
                  as_text=True)